In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
pd.set_option('display.max_columns', None)

In [2]:
# Initialize model and move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = SimpleNN().to(device)

Using device: cuda


In [32]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10212, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 2)  # Assuming you have 10 output classes
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

In [25]:
class MyDataset(Dataset):
    def __init__(self, pd_dataFrame):
        self.data = pd_dataFrame
        self.X = self.data.iloc[:, :-1].values  # all columns except last
        self.y = self.data.iloc[:, -1].values   # last column as label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

In [26]:
df = pd.read_csv('Trim_train/split_part_1.csv')
df = df.drop("ID", axis=1)
df = df.fillna(0)
train_dataset = MyDataset(df)
#test_dataset = MyDataset('test.csv')
# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [33]:
# Assuming your SimpleNN model and device setup from earlier
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [34]:
num_epochs = 10  # You can adjust this

for epoch in range(num_epochs):
    model.train()  # set model to training mode
    running_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/10], Loss: 0.1169
Epoch [2/10], Loss: 0.0561
Epoch [3/10], Loss: 0.0497
Epoch [4/10], Loss: 0.0407
Epoch [5/10], Loss: 0.0354
Epoch [6/10], Loss: 0.0271
Epoch [7/10], Loss: 0.0259
Epoch [8/10], Loss: 0.0258
Epoch [9/10], Loss: 0.0195
Epoch [10/10], Loss: 0.0086
